In [4]:
import tushare as ts
import matplotlib.pyplot as plt
import matplotlib.finance as mpf
import numpy as np
import talib as talib
import pandas as pd
import datetime as dt

#取所有2n天内的低点
def get_low_point(df,columns,n):
    conds = (df[columns]==df[columns])
    for i in range(0,n):
        conds = conds & (df[columns].shift(i+1)>=df[columns]) &  (df[columns].shift(-i-1)>=df[columns])  
    return df[conds] 
#取所有2n天内的高点
def get_height_point(df,col,n):
    conds = (df[columns]==df[columns])
    for i in range(0,n):
        conds = conds & (df[columns].shift(i+1)<=df[columns]) &  (df[columns].shift(-i-1)<=df[columns])  
    return df[conds]

data = ts.get_k_data('002023', ktype='m',autype="qfq",index=False,start='2001-01-01', end='2018-05-15')
#RSI
data["rsi"] = talib.RSI(data['close'].values, timeperiod=9)
 

In [9]:
low=data["low"]
low_Low= low[ (low<low.shift(3))&(low<=low.shift(2))& (low<=low.shift(1)) & (low<=low.shift(-1)) & (low<=low.shift(-2)) & (low<=low.shift(-3)) ] 
k_LowPoint= get_low_point(data,"low",3).dropna()
#画出低点
#plt.plot(low_Low.index.values,low_Low.values,'o',color='red')
k_LowPoint 
k_LowPoint

,date,open,close,high,low,volume,code,rsi
13,2005-08-31,1.986,1.739,2.014,1.665,535714.77,002023,41.103242
20,2006-03-31,1.739,1.655,1.783,1.563,273536.55,002023,40.480279
28,2006-11-30,2.217,1.997,2.259,1.835,311509.48,002023,47.118864
39,2007-10-31,3.536,2.750,3.563,2.500,258140.58,002023,48.874608
52,2008-11-28,1.109,1.302,1.509,1.082,485698.21,002023,34.627535
70,2010-05-31,6.842,5.890,6.921,4.866,500537.31,002023,59.803151
78,2011-01-31,8.093,6.973,8.352,6.096,628061.00,002023,53.775655
90,2012-01-31,4.727,3.707,4.779,3.419,420685.00,002023,27.813509
96,2012-07-31,4.347,3.778,4.479,3.749,497526.00,002023,32.583233
101,2012-12-31,4.002,4.571,4.591,3.744,907748.00,002023,46.077176
